In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import re 
import string

import nltk;
from nltk import untag
from nltk.collocations import *;
from nltk.tokenize import word_tokenize;
from nltk.corpus import stopwords;
from nltk.probability import FreqDist
from nltk.tokenize import TweetTokenizer
from nltk import ngrams

In [22]:
from nltk.corpus import wordnet
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/keithpallo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
pd.options.mode.chained_assignment = None  # default='warn'

In [5]:
nltk.download('averaged_perceptron_tagger');

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/keithpallo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [107]:
df_13 = pd.read_json("gg2013.json")
df_14 = pd.read_json("../gg2015.json")

In [108]:
OFFICIAL_AWARDS_1315 = ['cecil b. demille award', 'best motion picture - drama', 'best performance by an actress in a motion picture - drama', 'best performance by an actor in a motion picture - drama', 'best motion picture - comedy or musical', 'best performance by an actress in a motion picture - comedy or musical', 'best performance by an actor in a motion picture - comedy or musical', 'best animated feature film', 'best foreign language film', 'best performance by an actress in a supporting role in a motion picture', 'best performance by an actor in a supporting role in a motion picture', 'best director - motion picture', 'best screenplay - motion picture', 'best original score - motion picture', 'best original song - motion picture', 'best television series - drama', 'best performance by an actress in a television series - drama', 'best performance by an actor in a television series - drama', 'best television series - comedy or musical', 'best performance by an actress in a television series - comedy or musical', 'best performance by an actor in a television series - comedy or musical', 'best mini-series or motion picture made for television', 'best performance by an actress in a mini-series or motion picture made for television', 'best performance by an actor in a mini-series or motion picture made for television', 'best performance by an actress in a supporting role in a series, mini-series or motion picture made for television', 'best performance by an actor in a supporting role in a series, mini-series or motion picture made for television']


In [85]:
def clean_awards(text):
    " Cleans individual tweet for award search"
    
    remove_terms = ['#goldenglobes', 'golden globes', '#goldenglobe', 'golden globe', 'goldenglobes', 'goldenglobe', 'rt', 'golden', 'globe', 'globes']
    
    text = re.sub("(\s)#\w+","",text)    # strips away all hashtags 
    text = re.sub("RT","",text)          # removes retweet
    text = re.sub("[^a-zA-Z ]", '',text) # removes all punctuation but keeps whitespace for tokenization
    text = text.lower()
    text = text.split()
    text = " ".join([term for term in text if term not in remove_terms]) #remove stop words
    
    return text 

In [14]:
def find_tags(tweet):
    """
    Performs pos tagging at a tweet level
    """
    
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(tweet)
    tags = nltk.pos_tag(tokens)
    return tags

In [15]:
def pos_search(tags,chunk_gram,label):
    ""
    
    
    potentials = ""
    chunk_parser = nltk.RegexpParser(chunk_gram)
    chunked = chunk_parser.parse(tags)
    for subtree in chunked.subtrees():
        if subtree.label() == label: 
            raw_list = untag(subtree)
            raw_list = [i for i in raw_list if wordnet.synsets(i)]
            string = ' '.join(raw_list)
            if "best" in string[0:6]:
                if len(string) >= len(potentials):
                    potentials = string
                    
    if potentials == "":
        return "No Chunk"

    return potentials

In [16]:
def filter_df(df,label):
    
    data = df.loc[df[label] != "No Chunk"]
    data.drop(data.columns.difference([label]), 1, inplace=True)
    single_list = list(data[label])
    freq = FreqDist(single_list)
    
    return data, freq

In [102]:
def find_awards(df):
    """
    Returns a list of strings for all possible awards
    """
    # Shuffle data if necesarry
    sample_size = 200000
    if len(df['text']) > sample_size:
        df = df.sample(n=sample_size)
    
    # Clean awards, keep best, pos tag
    df['text'] = df['text'].apply(lambda x:  clean_awards(x))
    df_a = df[df['text'].str.contains("best")]
    df_a['tags'] = df_a['text'].apply(lambda x: find_tags(x))
    
    # Define regex patterns from generalized 
    regex_pattern_0 = "P0: {<JJ.><NN.|JJ|VBG><...?>*<NN.>}"
    regex_pattern_1 = "P1: {<NN.><IN|NN.|IN><...?>*<NN.>}"
    regex_pattern_2 = "P2: {<RB.><JJ|NN.|VGB><...?>*<NN.|JJ>}"
    
    # Search for pos 
    df_a['chunks_0'] = df_a['tags'].apply(lambda x: pos_search(x,regex_pattern_0,"P0"))
    df_a['chunks_1'] = df_a['tags'].apply(lambda x: pos_search(x,regex_pattern_1,"P1"))
    df_a['chunks_2'] = df_a['tags'].apply(lambda x: pos_search(x,regex_pattern_2,"P2"))

    data_0, freq_0 = filter_df(df_a,"chunks_0")
    data_1, freq_1 = filter_df(df_a,"chunks_1")
    data_2, freq_2 = filter_df(df_a,"chunks_2")

    freq = freq_0 + freq_1 + freq_2
    
    possible = []

    for i in freq.most_common():
        if i[1] >= 8: possible.append(i[0])
    
    return possible

In [104]:
possible = find_awards(df_13)


In [103]:
df_13 = pd.read_json("../gg2015.json")
df_13 = pd.DataFrame(df_13)
"""
sample_size = 200000
if len(df_13['text']) > sample_size:
    df_13 = df_13.sample(n=sample_size)
"""

"\nsample_size = 200000\nif len(df_13['text']) > sample_size:\n    df_13 = df_13.sample(n=sample_size)\n"

In [87]:
df_13['text'] = df_13['text'].apply(lambda x:  clean_awards(x))
df_a = df_13[df_13['text'].str.contains("best")]
df_a['tags'] = df_a['text'].apply(lambda x: find_tags(x))

In [90]:
df_a['chunks_0'] = df_a['tags'].apply(lambda x: pos_search(x,regex_pattern_0,"P0"))
df_a['chunks_1'] = df_a['tags'].apply(lambda x: pos_search(x,regex_pattern_1,"P1"))
df_a['chunks_2'] = df_a['tags'].apply(lambda x: pos_search(x,regex_pattern_2,"P2"))

data_0, freq_0 = filter_df(df_a,"chunks_0")
data_1, freq_1 = filter_df(df_a,"chunks_1")
data_2, freq_2 = filter_df(df_a,"chunks_2")

freq = freq_0 + freq_1 + freq_2

In [100]:
from pprint import pprint

# pprint(freq.most_common())

possible = []

for i in freq.most_common():
    if i[1] >= 8: possible.append(i[0])
        
pprint(possible)
pprint(len(possible))

['best supporting actor in a television series matt',
 'best supporting actress in a tv movie series or',
 'best dressed men',
 'best supporting actor in a motion picture',
 'best supporting actor in a series',
 'best supporting actress in a series',
 'best played girls',
 'best awards show monologue in history awards show monologues',
 'best actress musical or comedy at big eyes',
 'best dressed celebrities',
 'best original song at incredible words gentlemen',
 'best dresses suits',
 'best original song at awards',
 'best foreign film at read about russias',
 'best actress tv',
 'best speeches always come heart not a publicists',
 'best dressed stars',
 'best supporting actor in a motion picture goes',
 'best supporting actor',
 'best original song winners',
 'best opening monologue jokes',
 'best supporting actor goes',
 'best original song common',
 'best original song',
 'best dressed stars at awards naomi watts williams',
 'best worst amp wackiest dressed stars',
 'best supportin

In [101]:

#print(sum(freq.values()))

test = []

for i in freq.most_common(100):
    test.append(i[0])

    lt = i[0].split()
    string = ""
    for s in lt:
        if wordnet.synsets(s):
            string += s
            string += " "
        #print(string)


test_2 = []
simple_test = [" ".join(i.split()[:3]) for i in test]

pprint(test)

test_2.append(test[0])
for i in range(1,len(test)):
    num_words = test[i].split()
    if len(num_words) >= 3:
        string = " ".join(test[i].split()[:3])
        #print(string)
        if string not in simple_test[0:i] and "golden globes" not in test[i]:
            test_2.append(test[i])
      
#pprint(test_2)
    
    
    # Remove [golden globes ]
    # Cut after [] ?? 
    # Condense substrings (if longer than three substring is more frequent, then kick out)
    


['best supporting actor in a television series matt',
 'best supporting actress in a tv movie series or',
 'best dressed men',
 'best supporting actor in a motion picture',
 'best supporting actor in a series',
 'best supporting actress in a series',
 'best played girls',
 'best awards show monologue in history awards show monologues',
 'best actress musical or comedy at big eyes',
 'best dressed celebrities',
 'best original song at incredible words gentlemen',
 'best dresses suits',
 'best original song at awards',
 'best foreign film at read about russias',
 'best actress tv',
 'best speeches always come heart not a publicists',
 'best dressed stars',
 'best supporting actor in a motion picture goes',
 'best supporting actor',
 'best original song winners',
 'best opening monologue jokes',
 'best supporting actor goes',
 'best original song common',
 'best original song',
 'best dressed stars at awards naomi watts williams',
 'best worst amp wackiest dressed stars',
 'best supportin

In [135]:
df_13 = pd.read_json("gg2013.json")

In [186]:
df_13['text'] = df_13['text'].apply(lambda x:  clean_for_regex(x))


def possible_movies(text):
    #r1 = re.findall(r"([A-Z][a-z]+(?=\s[A-Z])(?:\s[A-Z][a-z]+)+)",text)
    #print(r1)
    #r2 = re.findall(r"(\b[A-Z].+?\b)([a-z].*)(\b[A-Z].+?\b)",text)
    #text = ' '.join([term.upper() for term in text if term.lower() in stopwords.words("english")])
    #r2 = re.findall(r"([A-Z][a-z]+(?=\s[A-Z])(?:\s[A-Z][a-z]+)+)",text)
    #print(r2)
    return r1

In [187]:
test_regex = list(df_13['text'])

In [2]:
#possible = []

#for i in test_regex:
#    possible.append(possible_movies(i))

In [1]:
# pprint(possible)

In [37]:
def simple_test(dic,freq):
    
    test = []
    
    for key in dic.keys():
        if key in freq:
            test.append([key,freq[key]])
        else:
            test.append([key,"Not found"])
        
    print(*test,sep='\n')   
    return test

In [140]:
"""

def clean_for_regex(text):
    text = re.sub("(\s)#\w+","",text)    # strips away all hashtags 
    text = re.sub("RT","",text)
    text = re.sub("[^a-zA-Z ]", '',text) # removes all punctuation but keeps whitespace for tokenization 
    #text = text.lower()
    #if "best" not in text:
    #    return ""
    
    return text 
    
"""